In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00


In [2]:
import random as ra

import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, GRU, SpatialDropout2D
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from keras.models import Model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import BertTokenizer, TFBertModel

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv("/content/drive/MyDrive/Movie Research/yelp_reviews.csv")

In [5]:
data

,text,stars
0,"If you decide to eat here, just be aware it is...",3.0
1,I've taken a lot of spin classes over the year...,5.0
2,Family diner. Had the buffet. Eclectic assortm...,3.0
3,"Wow! Yummy, different, delicious. Our favo...",5.0
4,Cute interior and owner (?) gave us tour of up...,4.0
...,...,...
6990275,Latest addition to services from ICCU is Apple...,5.0
6990276,"This spot offers a great, affordable east week...",5.0
6990277,This Home Depot won me over when I needed to g...,4.0
6990278,For when I'm feeling like ignoring my calorie-...,5.0


In [6]:
temp = data.sample(n = 350000, random_state=42)
df = temp.sample(frac = .1, random_state=42)

In [7]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00


In [8]:
from keras_preprocessing.sequence import pad_sequences

In [9]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [11]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

In [12]:
# df['text'] = df['text'].str.lower()

In [13]:
# df['text'] = df['text'].str.replace('[^\w\s]', '')

In [14]:
# Apply the remove_stopwords and lemmatize_text functions to the 'text' column
# df['text'] = df['text'].apply(remove_stopwords)
# df['text'] = df['text'].apply(lemmatize_text)

In [15]:
# train_df = df[:27000]
# test_df = df[27000:]

In [16]:
# max_features = 20000

In [17]:
# # Tokenization
# tokenizer = Tokenizer(num_words=max_features)
# tokenizer.fit_on_texts(list(train_df['text'].values))
# X_train = tokenizer.texts_to_sequences(train_df['text'].values)
# X_test = tokenizer.texts_to_sequences(test_df['text'].values)

In [18]:
# x_train = pad_sequences(X_train, maxlen=200)
# x_test = pad_sequences(X_test, maxlen=200)

In [19]:
# word_index = tokenizer.word_index
# nb_words = min(max_features, len(word_index))

In [20]:
# embedding_matrix = np.zeros((nb_words, 200))

In [21]:
# embedding_file = '/content/drive/MyDrive/Movie Research/glove.twitter.27B.200d.txt'

# read in embeddings
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
# embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(embedding_file, 'r', encoding = 'utf-8'))

In [ ]:
# word_index = tokenizer.word_index
# nb_words = min(max_features, len(word_index))

# embedding_matrix = np.zeros((nb_words, 200))

In [ ]:
# missed = []

In [ ]:
# for word, i in word_index.items():

#     if i == max_features:
#         break

#     embedding_vector = embeddings_index.get(word)

#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector
#     else:
#         missed.append(word)

In [22]:
max_features = 20000
def get_tokenizer(df):
  # Tokenization
  global max_features
  tokenizer = Tokenizer(num_words=max_features)
  tokenizer.fit_on_texts(list(df['text'].values))
  return tokenizer

def preprocess_dataframe(tokenizer, df, maxlen=200):
  df['text'] = df['text'].str.lower()
  df['text'] = df['text'].str.replace('[^\w\s]', '')
  # Apply the remove_stopwords and lemmatize_text functions to the 'text' column
  df['text'] = df['text'].apply(remove_stopwords)
  df['text'] = df['text'].apply(lemmatize_text)
  X = tokenizer.texts_to_sequences(df['text'].values)
  x = pad_sequences(X, maxlen=maxlen)
  return x


In [23]:
embedding_file = '/content/drive/MyDrive/Movie Research/glove.twitter.27B.200d.txt'
def get_embedding_matrix(tokenizer):
  global embedding_file
  word_index = tokenizer.word_index
  nb_words = min(max_features, len(word_index))
  embedding_matrix = np.zeros((nb_words, 200))
  embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(embedding_file, 'r', encoding = 'utf-8'))
  missed = []
  for word, i in word_index.items():

    if i == max_features:
        break

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        missed.append(word)

    return embedding_matrix

In [24]:
def make_predictions(model, df):
  x = preprocess_dataframe(df)
  return model.predict(x)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

def check_params(units, spatial_dropout_rate, batch_size, count):
  global df
  inp = Input(shape = (200,))
  tokenizer = get_tokenizer(df)
  embedding_matrix = get_embedding_matrix(tokenizer)
  x = Embedding(max_features, 200, weights = [embedding_matrix], trainable = True)(inp)
  x = SpatialDropout1D(spatial_dropout_rate)(x)
  x = Bidirectional(LSTM(units, return_sequences=True))(x)
  x = Bidirectional(LSTM(units, return_sequences=True))(x)
  x = Bidirectional(GRU(units, return_sequences=True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, max_pool])
  outp = Dense(5, activation = 'sigmoid')(conc)

  model = Model(inputs = inp, outputs = outp)
  # patience is how many epochs to wait to see if val_loss will improve again.
  earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3)
  checkpoint = ModelCheckpoint(monitor = 'val_loss', save_best_only = True, filepath = f'./models/yelp_lstm_gru_weights_{count}.hdf5')
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

  train_df = df[:27000]
  x_train = preprocess_dataframe(tokenizer, train_df)
  y_train = pd.get_dummies(train_df['stars'])

  history = model.fit(x_train, y_train, batch_size = batch_size, epochs = 10, validation_split = .2,
          callbacks=[earlystop, checkpoint])

  return history.history['accuracy']

def generate_random_number(x):
  return ra.randint(0,x-1)

def train_model_random_search(model_count):
  param_grid = {
      'units': [30, 60, 90],
      'spatial_dropout_rate': [0.3, 0.5],
      'batch_size': [256,512]
  }

  config_list = []

  for i in range(model_count):
    units = param_grid['units'][generate_random_number(len(param_grid['units']))]
    spatial_dropout_rate = param_grid['spatial_dropout_rate'][generate_random_number(len(param_grid['spatial_dropout_rate']))]
    batch_size = param_grid['batch_size'][generate_random_number(len(param_grid['batch_size']))]
    config_list.append((units,spatial_dropout_rate,batch_size))

    if((units,spatial_dropout_rate,batch_size) in config_list):
      --i

  results = dict()

  count = 0
  for i in config_list:
    count+=1
    results[i] = check_params(i[0], i[1], i[2], count)



  return results

In [ ]:
train_model_random_search(3)

<ipython-input-22-366570351d34>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()
<ipython-input-22-366570351d34>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]', '')
<ipython-input-22-366570351d34>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.replace('[^\w\s]', '')
<ipython-input-22-366570351d34>:13: SettingWithCopyWarning: 
A value is trying

Epoch 1/10
43/43 [==============================] - 666s 15s/step - loss: 0.4794 - accuracy: 0.4510 - val_loss: 0.4333 - val_accuracy: 0.4709
Epoch 2/10
43/43 [==============================] - 635s 15s/step - loss: 0.3794 - accuracy: 0.5442 - val_loss: 0.3371 - val_accuracy: 0.5952
Epoch 3/10
43/43 [==============================] - 632s 15s/step - loss: 0.3120 - accuracy: 0.6236 - val_loss: 0.3314 - val_accuracy: 0.6143
Epoch 4/10
43/43 [==============================] - 634s 15s/step - loss: 0.2778 - accuracy: 0.6753 - val_loss: 0.3033 - val_accuracy: 0.6400
Epoch 5/10
43/43 [==============================] - 650s 15s/step - loss: 0.2365 - accuracy: 0.7363 - val_loss: 0.3087 - val_accuracy: 0.6469
Epoch 6/10
43/43 [==============================] - 630s 15s/step - loss: 0.2051 - accuracy: 0.7812 - val_loss: 0.3302 - val_accuracy: 0.6259
Epoch 7/10
43/43 [==============================] - 638s 15s/step - loss: 0.1789 - accuracy: 0.8183 - val_loss: 0.3544 - val_accuracy: 0.6093


<ipython-input-22-366570351d34>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()
<ipython-input-22-366570351d34>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]', '')
<ipython-input-22-366570351d34>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.replace('[^\w\s]', '')
<ipython-input-22-366570351d34>:13: SettingWithCopyWarning: 
A value is trying

Epoch 1/10
85/85 [==============================] - 665s 8s/step - loss: 0.4334 - accuracy: 0.4887 - val_loss: 0.3368 - val_accuracy: 0.5994
Epoch 2/10
85/85 [==============================] - 678s 8s/step - loss: 0.3087 - accuracy: 0.6273 - val_loss: 0.2948 - val_accuracy: 0.6583
Epoch 3/10
85/85 [==============================] - 679s 8s/step - loss: 0.2560 - accuracy: 0.7036 - val_loss: 0.2966 - val_accuracy: 0.6567
Epoch 4/10
85/85 [==============================] - 685s 8s/step - loss: 0.2253 - accuracy: 0.7473 - val_loss: 0.3032 - val_accuracy: 0.6435
Epoch 5/10
47/85 [===============>..............] - ETA: 4:28 - loss: 0.1964 - accuracy: 0.7896

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM

# inp = Input(shape = (200,))
# x = Embedding(max_features, 200, weights = [embedding_matrix], trainable = True)(inp)
# x = SpatialDropout1D(0.3)(x)
# # x = SpatialDropout2D(0.3)(x)
# x = Bidirectional(LSTM(60, return_sequences=True))(x)
# x = Bidirectional(LSTM(60, return_sequences=True))(x)
# x = Bidirectional(GRU(60, return_sequences=True))(x)
# avg_pool = GlobalAveragePooling1D()(x)
# max_pool = GlobalMaxPooling1D()(x)
# conc = concatenate([avg_pool, max_pool])
# outp = Dense(5, activation = 'sigmoid')(conc)

# model = Model(inputs = inp, outputs = outp)
# # patience is how many epochs to wait to see if val_loss will improve again.
# earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3)
# checkpoint = ModelCheckpoint(monitor = 'val_loss', save_best_only = True, filepath = 'yelp_lstm_gru_weights.hdf5')
# model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# y_train = pd.get_dummies(train_df["stars"])

In [ ]:
# model = Model(inputs = inp, outputs = outp)
# # patience is how many epochs to wait to see if val_loss will improve again.

# earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3)
# checkpoint = ModelCheckpoint(monitor = 'val_loss', save_best_only = True, filepath = 'yelp_lstm_gru_weights.hdf5')
# model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size = 256, epochs = 10, validation_split = .1,
          callbacks=[earlystop, checkpoint])

In [ ]:
model.save('saved_model/sentiment_analyzer')

In [ ]:
store = tf.keras.models.load_model('saved_model/sentiment_analyzer')
store.summary()

In [ ]:
model.summary()

In [ ]:
model = model.load('./models/yelp_lstm_gru_weights_1.hdf5')
test_df = df[27000:]
make_predictions(model, test_df)